In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "/share/dutta/eyao/dataset/kaggle/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/share/dutta/eyao/dataset/kaggle'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory () that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import math
from scipy.stats import zscore
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
import tensorflow as tf
import warnings
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

### Exploring the data

In [ ]:
df = pd.read_csv('/share/dutta/eyao/dataset/kaggle/house-prices-advanced-regression-techniques/train.csv',index_col='Id')
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

# Data Cleaning
#### Dealing with missing values

In [ ]:
print("Percentage of Nan in each Column")
for column, percentage in ((df.isna().sum() / df.shape[0]) * 100).items():
    count = math.ceil(percentage)
    if count > 0:
        print(f"{column} : {count} %")

In [ ]:
df.drop(columns=['MasVnrType','PoolQC','PoolArea','BsmtHalfBath','KitchenAbvGr','Utilities'],inplace=True)

In [ ]:
df['LotFrontage'] = df['LotFrontage'].fillna(0)
df['Alley'] = df['Alley'].fillna("No Alley Acess")
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mean())
df['LotFrontage'] = df['LotFrontage'].fillna(0)
df['BsmtQual'] = df['BsmtQual'].fillna("No Basment")
df['BsmtCond'] = df['BsmtCond'].fillna("No Basment")
df['BsmtExposure'] = df['BsmtExposure'].fillna("No Basment")
df['BsmtFinType1'] = df['BsmtFinType1'].fillna("No Basment")
df['BsmtFinType2'] = df['BsmtFinType2'].fillna("No Basment")
df['Electrical'] = df['Electrical'].fillna(df['Electrical'].mode()[0])
df['FireplaceQu'] = df['FireplaceQu'].fillna("No Fireplace")
df['GarageType'] = df['GarageType'].fillna("No Garage")
df['GarageYrBlt'] = df['GarageYrBlt'].fillna(0)
df['GarageFinish'] = df['GarageFinish'].fillna("No Garage")
df['GarageQual'] = df['GarageQual'].fillna("No Garage")
df['GarageCond'] = df['GarageCond'].fillna("No Garage")
df['Fence'] = df['Fence'].fillna("No Fence")
df['MiscFeature'] = df['MiscFeature'].fillna("No Miscellaneous Feature")

#### Removing Outliers

In [ ]:
def remove_outlier(df, x):
    df[f'{x}_zscore'] = zscore(df[f'{x}'])
    df.drop(df[(df[f'{x}_zscore'] >= 3) | (df[f'{x}_zscore'] <= -3)].index, inplace=True)
    df.drop(columns=[f'{x}_zscore'], inplace=True)

In [ ]:
columns = df.select_dtypes('number').columns.tolist()
for column in columns:
    remove_outlier(df, column)

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.describe()

# Analysis and Visualization

In [ ]:
df.hist(figsize=(20, 20), xlabelsize=10, ylabelsize=10,color='#ffc0cb');

In [ ]:
plt.figure(figsize=(30,30))
sns.heatmap(df.select_dtypes('number').corr(),vmax=.8, annot=True,square=True,cmap='brg');

In [ ]:
MSSubClass = df.groupby('MSSubClass')['SalePrice'].mean()
MSSubClassMap ={20 : '1-STORY 1946 & NEWER ALL STYLES',
               30 : '1-STORY 1945 & OLDER',
               40 : '1-STORY W/FINISHED ATTIC ALL AGES',
               45 : "1-1/2 STORY - UNFINISHED ALL AGES",
                50 : "1-1/2 STORY FINISHED ALL AGES",
                60 : "2-STORY 1946 & NEWER",
                70 : "2-STORY 1945 & OLDER",
                75 : "2-1/2 STORY ALL AGES",
                80 : "SPLIT OR MULTI-LEVEL",
                85 : "SPLIT FOYER",
                90 : "DUPLEX - ALL STYLES AND AGES",
               120 : "1-STORY PUD (Planned Unit Development) - 1946 & NEWER",
               150 : "1-1/2 STORY PUD - ALL AGES",
               160 : "2-STORY PUD - 1946 & NEWER",
               180 : "PUD - MULTILEVEL - INCL SPLIT LEV/FOYER",
               190 : "2 FAMILY CONVERSION - ALL STYLES AND AGES"}

index_list = MSSubClass.index.tolist()
fig =px.bar(y=MSSubClass.values, x=[MSSubClassMap[i] for i in index_list], labels={'x': 'MSSubClass','y':'Value'},color=MSSubClass.values)
fig.update_layout(title='Mean of SalePrice for each MSSubClass',width=1300, 
    height=1000 )
fig.show()

In [ ]:
MSZoningMap={
    "A" : "Agriculture",
       "C (all)" : "Commercial",
       "FV" : "Floating Village Residential",
       "I" : "Industrial",
       "RH" : "Residential High Density",
       "RL" : "Residential Low Density",
       "RP" : "Residential Low Density Park", 
       "RM" : "Residential Medium Density"
}
index_list = df['MSZoning'].value_counts().index.tolist()

px.pie(values=df['MSZoning'].value_counts().values,names=[MSZoningMap[i] for i in index_list],title='Number of Houses with the Zonning Classification')

In [ ]:
def plot_numeric_column_with_price(ax, x):
    scatter = ax.scatter(x=df[x], y=df['SalePrice'], c=df['SalePrice'], cmap='viridis', alpha=0.8, s=10)
    ax.set(xlabel=x, ylabel='SalePrice', title=f'{x} With Sale Price')
    plt.colorbar(scatter, ax=ax, label='SalePrice')

columns = []
for c in df.select_dtypes('number').columns.tolist():
    if df[f'{c}'].nunique() > 16:
        columns.append(c)
        
num_rows = math.ceil(len(columns[:-1]) / 2)
fig, axes = plt.subplots(num_rows, 2, figsize=(18, 6 * num_rows))
axes = axes.flatten()
for i, column in enumerate(columns[:-1]):
    if i >= len(axes):
        break
    plot_numeric_column_with_price(axes[i], column)

plt.tight_layout()
plt.show()

In [ ]:
def plot_categorical_column_with_price(x, ax):
    mean = df.groupby(x)['SalePrice'].mean()
    sns.barplot(x=mean.index, y=mean.values, width=0.5, ax=ax,palette='viridis')
    ax.set_xlabel(x)
    ax.set_ylabel('SalePrice')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_title(f'Mean of Sale Price With {x}')

columns = df.select_dtypes(exclude='number').columns.tolist()
for c in df.select_dtypes('number').columns.tolist():
    if df[f'{c}'].nunique() <= 16:
        columns.append(c)

num_columns = 3
num_rows = (len(columns) + num_columns - 1) // num_columns

fig, axes = plt.subplots(num_rows, num_columns, figsize=(18, 6 * num_rows))

for i, column in enumerate(columns):
    row = i // num_columns
    col = i % num_columns
    if num_rows > 1:
        ax = axes[row, col]
    else:
        ax = axes[col]
    plot_categorical_column_with_price(column, ax)

for i in range(len(columns), num_rows * num_columns):
    row = i // num_columns
    col = i % num_columns
    if num_rows > 1:
        fig.delaxes(axes[row, col])
    else:
        fig.delaxes(axes[col])

plt.tight_layout()
plt.show()

In [ ]:
def box_plot_categorical_column_with_price(x, ax):
    sns.boxplot(x=x, y='SalePrice', data=df, ax=ax, palette='viridis')
    ax.set_xlabel(x)
    ax.set_ylabel('SalePrice')
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')
    ax.set_title(f'Box Plot of Sale Price by {x}')


columns = df.select_dtypes(exclude='number').columns.tolist()
for c in df.select_dtypes('number').columns.tolist():
    if df[f'{c}'].nunique() <= 16:
        columns.append(c)
        
num_columns = 3
num_rows = (len(columns) + num_columns - 1) // num_columns

fig, axes = plt.subplots(num_rows, num_columns, figsize=(18, 6 * num_rows), sharey=True)
axes = axes.flatten()

for i, column in enumerate(columns):
    box_plot_categorical_column_with_price(column, axes[i])

for i in range(len(columns), num_rows * num_columns):
    fig.delaxes(axes[i])

plt.tight_layout()
plt.show()

# Preprocessing

#### Ecoding the data


In [ ]:
df = pd.get_dummies(df,dtype=float,drop_first=True)

#### Split the data

In [ ]:
df.head()

In [ ]:
index_to_exclude = df.columns.get_loc('SalePrice')
X = df.iloc[:, [i for i in range(df.shape[1]) if i != index_to_exclude]].values
y = df.iloc[:,index_to_exclude].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=329) 

#### Apply Standardization on the data

In [ ]:
X_train_nstd = X_train.copy()
X_test_nstd = X_test.copy()
y_train_nstd = y_train.copy()

sc_X = StandardScaler()
X_train[:, :34] = sc_X.fit_transform(X_train[:, :34])
X_test[:, :34] = sc_X.transform(X_test[:, :34])
sc_y = StandardScaler()
y_train = sc_y.fit_transform(y_train.reshape(-1,1)).flatten()

# Multi-Linear Regression

#### Train the Model

In [ ]:
multi_lr = LinearRegression()
multi_lr.fit(X_train_nstd, y_train_nstd)

#### Evaluate the model

In [ ]:
y_pred = multi_lr.predict(X_test_nstd)
y_train_pred = multi_lr.predict(X_train_nstd)
r2_lr_train = r2_score(y_train_nstd, y_train_pred)
r2_lr_test = r2_score(y_test, y_pred.reshape(-1,1))
print("R2 Train Score:", r2_lr_train)
print("R2 Test Score:", r2_lr_test)
mse_lr_train = mean_squared_error(y_train_nstd, y_train_pred)
mse_lr_test = mean_squared_error(y_test, y_pred.reshape(-1,1))
print("Mean Squared Error of Train:", mse_lr_train)
print("Mean Squared Error of Test:", mse_lr_test)


# Polynomial Regression

#### Training the Model

In [ ]:
poly_reg = PolynomialFeatures(degree = 2)
X_train_poly = poly_reg.fit_transform(X_train)
X_test_poly = poly_reg.transform(X_test)
poly_lr = LinearRegression()
poly_lr.fit(X_train_poly, y_train)

#### Evaluation the Model

In [ ]:
y_pred = poly_lr.predict(X_test_poly)
y_train_pred = poly_lr.predict(X_train_poly)
r2_poly_train = r2_score(y_train,y_train_pred)
r2_poly_test = r2_score(y_test,sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_poly_train)
print("R2 Test Score:", r2_poly_test)
mse_poly_train = mean_squared_error(y_train, y_train_pred)
mse_poly_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_poly_train)
print("Mean Squared Error of Test:", mse_poly_test)

# Decision Tree

### Training the Model

In [ ]:
dec_tree = DecisionTreeRegressor()
dec_tree.fit(X_train, y_train)

#### Evaluation the Model

In [ ]:
y_pred = dec_tree.predict(X_test)
y_train_pred = dec_tree.predict(X_train)
r2_tree_train = r2_score(y_train, y_train_pred)
r2_tree_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_tree_train)
print("R2 Test Score:", r2_tree_test)
mse_tree_train = mean_squared_error(y_train, y_train_pred)
mse_tree_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_tree_train)
print("Mean Squared Error of Test:", mse_tree_test)

# Random Forest

#### Training the model

In [ ]:
rdm_frst = RandomForestRegressor(n_estimators = 100)
rdm_frst.fit(X_train, y_train)

#### Evaluation the model

In [ ]:
y_pred = rdm_frst.predict(X_test)
y_train_pred = rdm_frst.predict(X_train)
r2_frst_train = r2_score(y_train, y_train_pred)
r2_frst_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_frst_train)
print("R2 Test Score:", r2_frst_test)
mse_frst_train = mean_squared_error(y_train, y_train_pred)
mse_frst_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_frst_train)
print("Mean Squared Error of Test:", mse_frst_test)

# Linear SVR

#### Training the model

In [ ]:
svr = SVR(kernel = 'linear')
svr.fit(X_train, y_train)

#### Evaluation the model

In [ ]:
y_pred = svr.predict(X_test)
y_train_pred = svr.predict(X_train)
r2_svr_lr_train = r2_score(y_train, y_train_pred)
r2_svr_lr_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_svr_lr_train)
print("R2 Test Score:", r2_svr_lr_test)
mse_svr_lr_train = mean_squared_error(y_train, y_train_pred)
mse_svr_lr_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_svr_lr_train)
print("Mean Squared Error of Test:", mse_svr_lr_test)

# SVR

#### Training the model

In [ ]:
svr = SVR(kernel = 'rbf')
svr.fit(X_train, y_train)

#### Evaluation the model

In [ ]:
y_pred = svr.predict(X_test)
y_train_pred = svr.predict(X_train)
r2_svr_train = r2_score(y_train, y_train_pred)
r2_svr_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_svr_train)
print("R2 Test Score:", r2_svr_test)
mse_svr_train = mean_squared_error(y_train, y_train_pred)
mse_svr_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_svr_train)
print("Mean Squared Error of Test:", mse_svr_test)

# Ridge

#### Training the model

In [ ]:
ridge = Ridge()
ridge.fit(X_train, y_train)

#### Evaluation the model

In [ ]:
y_pred = ridge.predict(X_test)
y_train_pred = ridge.predict(X_train)
r2_ridge_train = r2_score(y_train, y_train_pred)
r2_ridge_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_ridge_train)
print("R2 Test Score:", r2_ridge_test)
mse_ridge_train = mean_squared_error(y_train, y_train_pred)
mse_ridge_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_ridge_train)
print("Mean Squared Error of Test:", mse_ridge_test)


# Lasso

#### Training the Model

In [ ]:
lasso = Lasso(alpha=0.001)
lasso.fit(X_train, y_train)

#### Evaluation the model

In [ ]:
y_pred = lasso.predict(X_test)
y_train_pred = lasso.predict(X_train)
r2_lasso_train = r2_score(y_train, y_train_pred)
r2_lasso_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_lasso_train)
print("R2 Test Score:", r2_lasso_test)
mse_lasso_train = mean_squared_error(y_train, y_train_pred)
mse_lasso_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_lasso_train)
print("Mean Squared Error of Test:", mse_lasso_test)

# XGBoost Regressor

#### Training the model

In [ ]:
xgb = XGBRegressor(learning_rate=0.2,gamma=0.2)
xgb.fit(X_train, y_train)

#### Evaluation the model

In [ ]:
y_pred = xgb.predict(X_test)
y_train_pred = xgb.predict(X_train)
r2_xgb_train = r2_score(y_train, y_train_pred)
r2_xgb_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_xgb_train)
print("R2 Test Score:", r2_xgb_test)
mse_xgb_train = mean_squared_error(y_train, y_train_pred)
mse_xgb_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_xgb_train)
print("Mean Squared Error of Test:", mse_xgb_test)

# Neural Network

### Training the model

In [ ]:
tf.random.set_seed(329)
np.random.seed(329)

nn_model = tf.keras.Sequential([
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Dense(1)
])

nn_model.compile(loss = tf.keras.losses.mae,
                optimizer=tf.keras.optimizers.Adam(lr=0.1),
                metrics=['mae','mse'])

history = nn_model.fit(X_train,y_train,epochs=200,verbose=2)

In [ ]:
nn_model.summary()

#### Evaluating the model

In [ ]:
pd.DataFrame(history.history).plot()
plt.title('Loss Graph')
plt.ylabel('loss')
plt.xlabel('epochs');

In [ ]:
y_pred = nn_model.predict(X_test)
y_train_pred = nn_model.predict(X_train)
r2_nn_train = r2_score(y_train, y_train_pred)
r2_nn_test = r2_score(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("R2 Train Score:", r2_nn_train)
print("R2 Test Score:", r2_nn_test)
mse_nn_train = mean_squared_error(y_train, y_train_pred)
mse_nn_test = mean_squared_error(y_test, sc_y.inverse_transform(y_pred.reshape(-1,1)))
print("Mean Squared Error of Train:", mse_nn_train)
print("Mean Squared Error of Test:", mse_nn_test)

# Comparing Models

In [ ]:
models = pd.DataFrame({
    'Model': [
        'Multiple Linear Regression','Polynomial Regression','Decision Tree',
        'Random Forest', 'Support Vector Regression','Linear Support Vector Regression','Ridge','Lasso','XGBoost','Neural Network'
    ],
    'Training R2 Score': [
        r2_lr_train,r2_poly_train,r2_tree_train,r2_frst_train,r2_svr_train,r2_svr_lr_train,r2_ridge_train,r2_lasso_train,r2_xgb_train,r2_nn_train
    ],
    'Training Mean Square Error': [
       mse_lr_train,mse_poly_train,mse_tree_train,mse_frst_train,mse_svr_train,mse_svr_lr_train,mse_ridge_train,mse_lasso_train,mse_xgb_train,mse_nn_train
    ],
    'Testing R2 Score': [
        r2_lr_test,r2_poly_test,r2_tree_test,r2_frst_test,r2_svr_test,r2_svr_lr_test,r2_ridge_test,r2_lasso_test,r2_xgb_test,r2_nn_test
    ],
    'Testing Mean Square Error': [
       mse_lr_test,mse_poly_test,mse_tree_test,mse_frst_test,mse_svr_test,mse_svr_lr_test,mse_ridge_test,mse_lasso_test,mse_xgb_test,mse_nn_test
    ]
})

In [ ]:
models.sort_values(by='Testing R2 Score', ascending=False).style.background_gradient(
        cmap='Blues')

# If you like it Upvote it 